In [160]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

In [161]:
# Загрузка и обработка данных событий
from pathlib import Path
 
df_events = pd.DataFrame()
pathlist = Path('raw_data').glob('*.xlsm')
for path in pathlist:
    ds_events = pd.read_excel(path,sheet_name='Все события')
    # Удаляем лишние столбцы
    del(ds_events['serialno_a'],ds_events['id'],ds_events['location'],ds_events['site_address_b'],ds_events['site_address_c'],ds_events['type_full'])
    #Очищаем от событий с NaN
    ds_events.drop(ds_events['Объект'][ ds_events['Длительность'].isnull()].index,inplace=True)
    ds_events.drop(ds_events['Объект'][ ds_events['type'].isnull()].index,inplace=True)
    ds_events.drop(ds_events['Объект'][ ds_events['ТУ'].isnull()].index,inplace=True)
    #Очищаем от лишних событий 
    ds_events.drop(ds_events['Объект'][ ds_events['Важность']=='1 Информация'].index,inplace=True)
    ds_events.drop(ds_events['Объект'][ ds_events['type']=='Датчик открытия двери'].index,inplace=True)
    ds_events.drop(ds_events['Объект'][ ds_events['type']=='Прочее'].index,inplace=True)
    # Исторические замены
    ds_events.loc[ds_events['site_address_a']=='ДГУ - Сеть','site_address_a'] = 'ДГУ - Океанский'
    ds_events.loc[ds_events['site_address_a']=='ДГУ - ИТС','site_address_a'] = 'ДГУ - Океанский'
    ds_events.loc[ds_events['type']=='ОС Windows','type'] = 'ОС'
    ds_events.loc[ds_events['type']=='Витруализация','type'] = 'СКСД'
    ds_events.loc[ds_events['Событие']=='[#Manual] PortSecurity - Заблокирован порт','type'] = 'ИБ'
    # Меняем типы данных
    ds_events['Важность'] = ds_events['Важность'].apply(lambda x: x[:1])
    ds_events['Важность'] = ds_events['Важность'].astype(int)
    ds_events['Длительность'] = ds_events['Длительность'].astype(int)
    df_events = pd.concat([df_events,ds_events])

df_events.rename(columns = {'hardware':'model', 'site_address_a':'address', 'Событие':'event', 'Важность':'level', 'Дата':'start', 'Длительность':'duration', 'Объект':'object', 'ТУ':'tu', 'Востановление':'end'}, inplace = True )



df_events.loc[df_events.event.str.find('Изменилось состояние диска') != -1,'event'] = 'Сигнализация предупреждения на диске'
df_events.loc[df_events.event.str.find('Изменилось состояние') != -1,'event'] = 'Сигнализация предупреждения на оборудовании'
df_events.loc[df_events.event.str.find('неоптимальном состоянии') != -1,'event'] = 'Сигнализация предупреждения на оборудовании'
df_events.loc[df_events.event.str.find('DHCP') != -1,'event'] = 'Ошибка на DHCP'
df_events.loc[df_events.event.str.find('Zabbix') != -1,'event'] = 'Ошибка Zabbix'

df_events.loc[df_events.event.str.find('недоступ') != -1,'tag'] = 'available'
df_events.loc[df_events.event.str.find('задержк') != -1,'tag'] = 'perfomance'
df_events.loc[df_events.event.str.find('загрузк') != -1,'tag'] = 'perfomance'
df_events.loc[df_events.event.str.find('емператур') != -1,'tag'] = 'enviroment'
df_events.loc[df_events.event.str.find('тока') != -1,'tag'] = 'power'
df_events.loc[df_events.event.str.find('напряжение') != -1,'tag'] = 'power'
df_events.loc[df_events.event.str.find('нагрузка на ИБП') != -1,'tag'] = 'power'
df_events.loc[df_events.event.str.find('гипервиз') != -1,'tag'] = 'vm'
df_events.loc[df_events.event.str.find('протечк') != -1,'tag'] = 'enviroment'
df_events.loc[df_events.event.str.find('аканчив') != -1,'tag'] = 'available'
df_events.loc[df_events.event.str.find('igh') != -1,'tag'] = 'perfomance'
df_events.loc[df_events.event.str.find('устрой') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('данн') != -1,'tag'] = 'available'
df_events.loc[df_events.event.str.find('PING') != -1,'tag'] = 'network'
df_events.loc[df_events.event.str.find('порт') != -1,'tag'] = 'network'
df_events.loc[df_events.event.str.find('оборуд') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('работает от батар') != -1,'tag'] = 'ext_power'
df_events.loc[df_events.event.str.find('давлен') != -1,'tag'] = 'aspt'
df_events.loc[df_events.event.str.find('возду') != -1,'tag'] = 'enviroment'
df_events.loc[df_events.event.str.find('сточник питан') != -1,'tag'] = 'power'
df_events.loc[df_events.event.str.find('вария') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('критич') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('остоян') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('Общий статус - Предупреждение') != -1,'tag'] = 'vm'
df_events.loc[df_events.event.str.find('Disk space') != -1,'tag'] = 'available'
df_events.loc[df_events.event.str.find('заряд батареи') != -1,'tag'] = 'available'
df_events.loc[df_events.event.str.find('замена батарей') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('critical state') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('Порт отключен') != -1,'tag'] = 'network'
df_events.loc[df_events.event.str.find('Потеряно сетевое') != -1,'tag'] = 'network'
df_events.loc[df_events.event.str.find('Кластер АПКШ') != -1,'tag'] = 'network'
df_events.loc[df_events.event.str.find('конденсат') != -1,'tag'] = 'enviroment'

df_events.loc[df_events.event.str.find('Сигнализация ошибки на ИБП') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('Ошибка самотестирования') != -1,'tag'] = 'hardware'
df_events.loc[df_events.event.str.find('Пуск дизеля') != -1,'tag'] = 'ext_power'
df_events.loc[df_events.tag.isnull(), 'tag'] = 'other'



df_events.to_excel('events.xlsx',index=False)
df_events

,type,model,address,event,level,start,duration,object,tu,end,tag
0,ИБ,Cisco Catalyst C2960X,ДГУ - Океанский,[#Manual] PortSecurity - Заблокирован порт,2,2024-01-23 15:10:02,85803,0-3-3-C2960X,ДГУ,2024-01-24 15:00:05,network
1,ИБ,Cisco Catalyst C2960X,ДГУ - Океанский,[#Manual] PortSecurity - Заблокирован порт,2,2024-01-23 15:08:34,85891,0-3-3-C2960X,ДГУ,2024-01-24 15:00:05,network
2,ИБ,Cisco Catalyst C2960X,ДГУ - Океанский,[#Manual] PortSecurity - Заблокирован порт,2,2024-01-22 14:44:22,68143,0-3-3-C2960X,ДГУ,2024-01-23 09:40:05,network
3,ИБ,Cisco Catalyst C2960X,ДГУ - Океанский,[#Manual] PortSecurity - Заблокирован порт,2,2024-01-15 11:59:21,7844,0-3-3-C2960X,ДГУ,2024-01-15 14:10:05,network
4,ИБ,Cisco Catalyst C2960X,ДГУ - Океанский,[#Manual] PortSecurity - Заблокирован порт,2,2024-01-15 11:59:16,7849,0-3-3-C2960X,ДГУ,2024-01-15 14:10:05,network
...,...,...,...,...,...,...,...,...,...,...,...
3180,СКСД,Vmware vCenter 5.5,Якутия - Якутск,Высокая загрузка памяти,2,2024-03-29 15:50:56,132600,Якутия - Якутск / СКСД (S98VCENTER),Якутия,2024-03-31 04:40:56,perfomance
3181,СКСД,Vmware vCenter 5.5,Якутия - Якутск,Высокая загрузка памяти,2,2024-03-29 15:50:54,132600,Якутия - Якутск / СКСД (S98VCENTER),Якутия,2024-03-31 04:40:54,perfomance
3182,СКСД,Vmware vCenter 5.5,Якутия - Якутск,Индикация предупреждения на гипервизоре,2,2024-03-29 15:46:35,12,Якутия - Якутск / СКСД (S98VCENTER),Якутия,2024-03-29 15:46:47,vm
3185,СКСД,Vmware vCenter 5.5,Якутия - Якутск,Индикация предупреждения на гипервизоре,2,2024-03-29 15:46:32,12,Якутия - Якутск / СКСД (S98VCENTER),Якутия,2024-03-29 15:46:44,vm


In [162]:
# Загружаем и обрабатываем данные по объектам
df_object = pd.read_excel('raw_data/Объекты2.xlsx')
df_object.drop(df_object.loc[df_object['ZBX-Имя'].isnull()].index,inplace=True)
df_object.loc[df_object['Статус']=='Подключено','Статус'] = 'Подключен'
df_object.drop(df_object.loc[df_object['Статус'] != 'Подключен' ].index,inplace=True)
del(df_object['Статус'])

df_object.rename(columns = {'ZBX-Имя':'object', 'ТУ':'tu',  'Здание':'address', 'Тип объекта':'type', 'Модель':'model', 'Дата изменения':'date_add'}, inplace = True )
df_object.loc[df_object['address']=='ДГУ - Сеть','address'] = 'ДГУ - Океанский'
df_object.loc[df_object['address']=='ДГУ - ИТС','address'] = 'ДГУ - Океанский'
df_object.to_excel('objects.xlsx',index=False)
df_object

,object,tu,address,type,model,date_add
1,Анадырь - Дежнева / Сервер (s77srkias01),Анадырь,Анадырь - Дежнева,Физический сервер,HP DL380 G7,2024-03-28 00:00:00
2,Анадырь - Дежнева / Сервер (s77srkiod01),Анадырь,Анадырь - Дежнева,Физический сервер,HP DL380 G7,2024-03-28 00:00:00
3,Анадырь - Дежнева / Сервер (S77SRKUPR01),Анадырь,Анадырь - Дежнева,Физический сервер,HP DL380 G7,2024-03-28 00:00:00
4,Анадырь - Дежнева / Ленточная библиотека (S77S...,Анадырь,Анадырь - Дежнева,Ленточная библиотека,HP D2D4009i,2024-03-28 00:00:00
5,Анадырь - Дежнева / Ленточная библиотека (S77S...,Анадырь,Анадырь - Дежнева,Ленточная библиотека,HP D2D4009i,2024-03-28 00:00:00
...,...,...,...,...,...,...
1332,ДГУ - ЛВС - Светланская 71 / C3550-48_01,ДГУ,ДГУ - Океанский,Прочее,Узел сети,2023-07-01 19:08:32
1333,ДГУ - ЛВС - Светланская 71 / c3548XL_02_TND48,ДГУ,ДГУ - Океанский,Прочее,Узел сети,2023-07-01 19:08:32
1335,ДГУ - Светланская 71 / 17 (iNode),ДГУ,ДГУ - Светланская 71,Web/SNMP - адаптер,iNode-Light,2023-07-01 19:08:32
1339,ДГУ - Светланская 111 / Серверная (ИБП 1),ДГУ,ДГУ - Светланская 111,ИБП,APC Smart-UPS,2023-07-01 19:08:32
